In [25]:
%pip install pandas pyarrow diffusers torch safetensor transformers torchvision torchaudio sklearn accelerate==0.26.0


  Using cached diffusers-0.32.2-py3-none-any.whl.metadata (18 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement safetensor (from versions: none)
ERROR: No matching distribution found for safetensor


In [2]:
import torch 
torch.cuda.is_available()

True

In [27]:
import pandas as pd
import pyarrow
from sklearn.model_selection import train_test_split

# Read the Parquet file using the default (pyarrow) engine
df_train = pd.read_parquet('data/ARCChallenge/train/train-00000-of-00001.parquet', engine='pyarrow')
df_test = pd.read_parquet('data/ARCChallenge/test/test-00000-of-00001.parquet', engine='pyarrow')
df_val = pd.read_parquet('data/ARCChallenge/val/validation-00000-of-00001.parquet', engine='pyarrow')
df = pd.concat([df_train, df_test, df_val], axis=0, ignore_index=True)
df['answerKey'] = df['answerKey'].astype('string')

In [28]:
def create_prompt(question: str, choice_data: dict) -> str:
    """
    Create a prompt from a question string and a dictionary containing
    the choice labels and choices.

    Parameters:
        question (str): The question text.
        choice_data (dict): A dictionary with the keys:
            - "question_labels": a list of labels (e.g., ["A", "B", "C", "D"])
            - "choices": a list of choice strings.

    Returns:
        str: The formatted prompt.
    """
    # Retrieve labels and choices from the dictionary
    labels = choice_data.get("label", [])
    choices = choice_data.get("text", [])
    
    # Start building the prompt with the question string.
    prompt = f"Question: {question}\n\nChoices:\n"
    
    # If labels are provided and match the number of choices, use them.
    for label, choice in zip(labels, choices):
        prompt += f"{label}. {choice}\n"
    
    # Add cue for the model to provide an answer.
    prompt += "\nAnswer:"
    return prompt

In [29]:
df["prompt"] = df.apply(lambda row: create_prompt(row["question"], row["choices"]), axis=1)

In [30]:
for prompt in df["prompt"]:
    print(prompt)
    print("-" * 40)

Question: George wants to warm his hands quickly by rubbing them. Which skin surface will produce the most heat?

Choices:
A. dry palms
B. wet palms
C. palms covered with oil
D. palms covered with lotion

Answer:
----------------------------------------
Question: Which of the following statements best explains why magnets usually stick to a refrigerator door?

Choices:
A. The refrigerator door is smooth.
B. The refrigerator door contains iron.
C. The refrigerator door is a good conductor.
D. The refrigerator door has electric wires in it.

Answer:
----------------------------------------
Question: A fold observed in layers of sedimentary rock most likely resulted from the

Choices:
A. cooling of flowing magma.
B. converging of crustal plates.
C. deposition of river sediments.
D. solution of carbonate minerals.

Answer:
----------------------------------------
Question: Which of these do scientists offer as the most recent explanation as to why many plants and animals died out at the en

In [ ]:
df

,id,question,choices,answerKey,prompt
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': ['dry palms', 'wet palms', 'palms cov...",A,Question: George wants to warm his hands quick...
1,MCAS_2009_5_6516,Which of the following statements best explain...,"{'text': ['The refrigerator door is smooth.', ...",B,Question: Which of the following statements be...
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': ['cooling of flowing magma.', 'conver...",B,Question: A fold observed in layers of sedimen...
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': ['worldwide disease', 'global mountai...",D,Question: Which of these do scientists offer a...
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': ['west', 'east', 'north', 'south'], '...",B,Question: A boat is acted on by a river curren...
...,...,...,...,...,...
2585,Mercury_7090598,Which of these processes involves the transfer...,"{'text': ['erosion', 'sedimentation', 'subduct...",C,Question: Which of these processes involves th...
2586,OHAT_2007_5_24,"In a forest, how do decomposers help other org...",{'text': ['They release oxygen into the air th...,B,"Question: In a forest, how do decomposers help..."
2587,Mercury_SC_402239,What is the best way to conserve natural resou...,"{'text': ['Throw all glass in the trash.', 'Us...",C,Question: What is the best way to conserve nat...
2588,Mercury_7245088,Which describes the composition of carbohydrates?,{'text': ['lipids bonding to form phospholipid...,D,Question: Which describes the composition of c...


In [ ]:
import copy
from jinja2 import Template

def create_n_shot_prompt(df):
    prompt_list = []
    prompt_template = 'Given the following question and four candidate answers (A, B, C and D), choose the best answer.\nQuestion: {{question.strip()}}\nA. {{choices.text[0]}}\nB. {{choices.text[1]}}\nC. {{choices.text[2]}}{% if choices.text|length > 3 %}\nD. {{choices.text[3]}}{% endif %}\nYour response should end with "The best answer is [the_answer_letter]" where the [the_answer_letter] is one of A, B, C or D.'
    template = Template(prompt_template)
    for _, row in df.iterrows():
        rendered_prompt = template.render(question=row["question"], choices=row["choices"])
        prompt_list.append(rendered_prompt)
    return prompt_list

In [93]:
n_shot_prompt_list = create_n_shot_prompt(df_test, 0)

In [94]:
n_shot_prompt_list[0]

'Given the following question and four candidate answers (A, B, C and D), choose the best answer.\nQuestion: An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?\nA. Planetary density will decrease.\nB. Planetary years will become longer.\nC. Planetary days will become shorter.\nD. Planetary gravity will become stronger.\nYour response should end with "The best answer is [the_answer_letter]" where the [the_answer_letter] is one of A, B, C or D.'

In [35]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

torch.cuda.is_available()

True

In [36]:
!nvidia-smi

Tue Apr  8 08:59:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.83                 Driver Version: 572.83         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA TITAN X (Pascal)      WDDM  |   00000000:6B:00.0  On |                  N/A |
| 23%   37C    P5             22W /  250W |    4313MiB /  12288MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [37]:
import os

os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [38]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [39]:
%pip install transformers accelerate

Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import accelerate

CUDA_LAUNCH_BLOCKING=1
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", use_safetensors=True, device_map="auto", low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
# model = AutoModelForCausalLM.from_pretrained("C:/Users/Augustine/models/meta-llama--Llama-3_2-1B-Instruct", use_safetensors=True, device_map="auto", torch_dtype=torch.float16, low_cpu_mem_usage=True)
# tokenizer = AutoTokenizer.from_pretrained("C:/Users/Augustine/models/meta-llama--Llama-3_2-1B-Instruct")

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct.
401 Client Error. (Request ID: Root=1-67f4ede3-3b6b180b3229c9c201d9a5fa;c4c7aec6-e1fc-4646-ac47-d440aecff7b1)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-3B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [42]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model.generation_config.pad_token_id = tokenizer.pad_token_id

In [43]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

torch.cuda.is_available()

True

In [1]:
from tqdm import tqdm

def generate_answers(prompts_list):

    answer_list = []

    for prompt in tqdm(prompts_list):

        # formatted_prompt = tokenizer.apply_chat_template(prompt, add_generation_prompt=True, tokenize=False)
        inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(device)
        attention_mask = inputs['attention_mask']

        with torch.no_grad():
            outputs = model.generate(inputs['input_ids'],
                                    attention_mask=attention_mask,
                                    max_new_tokens=22,
                                    do_sample=True,
                                    pad_token_id=tokenizer.pad_token_id,
                                    eos_token_id=tokenizer.eos_token_id)
            
        prompt_length = inputs['input_ids'].shape[1]

        answer = tokenizer.decode(outputs[0][prompt_length:], skip_special_tokens=True)
        
        answer_list.append(answer)
    
    return answer_list

In [46]:
tokenizer.chat_template = "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [102]:
n_shot_prompt_list

['Given the following question and four candidate answers (A, B, C and D), choose the best answer.\nQuestion: An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?\nA. Planetary density will decrease.\nB. Planetary years will become longer.\nC. Planetary days will become shorter.\nD. Planetary gravity will become stronger.\nYour response should end with "The best answer is [the_answer_letter]" where the [the_answer_letter] is one of A, B, C or D.',
 'Given the following question and four candidate answers (A, B, C and D), choose the best answer.\nQuestion: A group of engineers wanted to know how different building designs would respond during an earthquake. They made several models of buildings and tested each for its ability to withstand earthquake conditions. Which will most likely result from testing different building designs?\nA. buildings will be built faster\nB. buildings will be made safer\nC.

In [104]:
answer_list = generate_answers(n_shot_prompt_list)

100%|██████████| 1172/1172 [06:45<00:00,  2.89it/s]


[' The best answer is C.',
 ' The best answer is B. The best answer is B.',
 ' The best answer is B.',
 ' The best answer is D.',
 ' The best answer is C. The best answer is C. The best answer is C.',
 ' The best answer is D.',
 ' The best answer is C.',
 ' The best answer is A.',
 ' The best answer is A.',
 ' The best answer is A.',
 ' The best answer is C.',
 ' The best answer is C.',
 ' The best answer is A.',
 ' The best answer is C.',
 ' The best answer is D.',
 ' The best answer is B.',
 ' The best answer is C.',
 ' The best answer is D.',
 ' The best answer is D.',
 ' The best answer is D.',
 ' The best answer is B.',
 ' The best answer is B.',
 ' The best answer is A.',
 ' The best answer is A.',
 ' The best answer is B.',
 ' The best answer is D.',
 ' The best answer is D.',
 ' The best answer is D.',
 ' The best answer is D.',
 ' The best answer is B.',
 ' The best answer is D.',
 ' The best answer is D.',
 ' The best answer is A.',
 ' The best answer is C.',
 ' The best answ

In [112]:
temp = []
for answer in answer_list:
    temp.append(answer[20])

match_list = [i for i, j in zip(temp, df_test['answerKey']) if i == j]
print(len(match_list)/len(df_test['answerKey']))


0.5349829351535836


In [109]:
temp

['C',
 'B',
 'B',
 'D',
 'C',
 'D',
 'C',
 'A',
 'A',
 'A',
 'C',
 'C',
 'A',
 'C',
 'D',
 'B',
 'C',
 'D',
 'D',
 'D',
 'B',
 'B',
 'A',
 'A',
 'B',
 'D',
 'D',
 'D',
 'D',
 'B',
 'D',
 'D',
 'A',
 'C',
 'A',
 'C',
 'D',
 'C',
 'C',
 'A',
 'C',
 'D',
 'C',
 'D',
 'B',
 'B',
 'C',
 'C',
 'D',
 'B',
 'A',
 'C',
 'C',
 'A',
 'A',
 'D',
 'D',
 'A',
 'B',
 'C',
 'B',
 'B',
 'D',
 'C',
 'D',
 'A',
 'B',
 'C',
 'D',
 'B',
 'B',
 'C',
 'A',
 'D',
 'D',
 'C',
 'B',
 'A',
 'B',
 'B',
 'D',
 'A',
 'A',
 'C',
 'C',
 'B',
 'A',
 'A',
 'A',
 'C',
 'A',
 'A',
 'A',
 'D',
 'B',
 'A',
 'C',
 'B',
 'B',
 'B',
 'C',
 'D',
 'B',
 'B',
 'B',
 'B',
 'D',
 'B',
 'D',
 'D',
 'C',
 'C',
 'C',
 'A',
 'B',
 'D',
 'C',
 'C',
 'C',
 'C',
 'A',
 'A',
 'B',
 'D',
 'A',
 'D',
 'A',
 'A',
 'D',
 'A',
 'D',
 'C',
 'A',
 'B',
 'D',
 'B',
 'D',
 'B',
 'A',
 'A',
 'D',
 'C',
 'A',
 'C',
 'A',
 'B',
 'C',
 'C',
 'C',
 'A',
 'C',
 'D',
 'D',
 'C',
 'D',
 'D',
 'A',
 'C',
 'B',
 'B',
 'C',
 'D',
 'C',
 'A',
 'A',
 'C',
 'C'

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-3B-Instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
from tqdm import tqdm

def pipeline_input_prompt_list(n_shot_prompt_list):
    output_list = []
    for prompt in tqdm(n_shot_prompt_list):
        generated_text = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id, batch_size=8)
        output_list.append(generated_text)
    
    return output_list

In [ ]:
output_list = pipeline_input_prompt_list(n_shot_prompt_list)

100%|██████████| 2590/2590 [12:16:26<00:00, 17.06s/it]  


In [ ]:
output_list

NameError: name 'output_list' is not defined

In [ ]:
for s in output_list:
    rest, n = s.rsplit('Answer: ', 1)

NameError: name 'output_list' is not defined